In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import s3fs
import yaml
import numpy as np
import fireducks.pandas as pd
import warnings
from dotenv import load_dotenv
from tqdm import tqdm
from pprint import pprint
import joblib

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import (
                                    cross_val_score,
                                    RandomizedSearchCV,
                                    GridSearchCV, 
                                    train_test_split,
                                    StratifiedKFold
                                    )
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from xgboost import XGBClassifier
from gensim.models import Word2Vec, FastText

sys.path.append("../src")
from ml_utils import *

ImportError: cannot import name 'triu' from 'scipy.linalg' (/usr/local/lib/python3.12/site-packages/scipy/linalg/__init__.py)

In [ ]:
load_dotenv()
pd.set_option("display.max_columns", None)
warnings.simplefilter("ignore")
fs = s3fs.S3FileSystem(
            client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"},
            key=os.environ["Accesskey"],
            secret=os.environ["Secretkey"],
            token=os.environ["Token"]
)

# Data Preprocessing

In [ ]:
with fs.open("elissamim/text_classification_men/data/stages-votes.json", "r") as file:
    df = pd.read_json(file)

df = df.groupby("phrase_text", as_index = False)["sol"].apply(lambda x: x.mode().iloc[0])
df["sol"]=df["sol"].apply(lambda x: 1 if x == "ok" else 0)
df["clean_phrase_text"] = df["phrase_text"].apply(lambda x: nltk_text_preprocessing(x, True))
df = df[df["clean_phrase_text"] != ""]
df.head()

# Model selection (static embedding (sparse or dense) + classification algorithm) with grid search and cross validation

In [ ]:
# Separate validation data for later
X = df["clean_phrase_text"]
y = df["sol"]

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size= 0.2,
                                                    stratify=y,
                                                    random_state = 42)

tokenized_texts = [text.split() for text in X_train]
word2vec_model = Word2Vec(sentences = tokenized_texts,
                         vector_size = 100,
                         window = 5,
                         min_count = 1,
                         workers = 4,
                         seed = 42)
fasttext_model = FastText(sentences = tokenized_texts,
                         vector_size = 100,
                         window = 5,
                         min_count = 1,
                         workers = 4,
                         seed = 42)

static_embedding_models = {
    # Sparse embeddings
    "Bag of Words":CountVectorizer(),
    "TF":TfidfVectorizer(use_idf=False),
    "TF-IDF":TfidfVectorizer(),
    # Dense embeddings
    "Word2Vec": MeanEmbeddingVectorizer(model=word2vec_model),
    "FastText": MeanEmbeddingVectorizer(model=fasttext_model)
}

classification_models = {
    "Logistic Regression":LogisticRegression(),
    "Random Forest":RandomForestClassifier(),
    "Linear SVM":SVC(kernel="linear", probability=True),
    "Multinomial Naive Bayes":MultinomialNB(),
    "XGBoost":XGBClassifier(use_label_encoder=False, eval_metric="logloss")
}

# Import hyperparameter grid for tuning models
with open("../configs/ml_hyperparameters.yaml", "r", encoding = "utf-8") as file:
    params_grid = yaml.load(file, Loader = yaml.FullLoader)

In [ ]:
dict_scores, dict_params = {}, {}

for embedding_name, embedding_model in tqdm(static_embedding_models.items(),
                                           desc="Static embeddings"):

    dict_scores[embedding_name], dict_params[embedding_name] = {}, {}
    
    for classification_name, classification_model in tqdm(classification_models.items(),
                                                         desc="Classification algorithms"):

        # Multinomial NB is not suited for dense vectors
        if embedding_name in ["Word2Vec", "FastText"] and classification_name == "Multinomial Naive Bayes":
            continue

        steps = [("feature_extraction", embedding_model)]

        # For Logistic Regression and Linear SVM, and for dense embeddings, add standardisation
        if embedding_name in ["Word2Vec", "FastText"] and classification_name in ["Logistic Regression", "Linear SVM"]:
            steps.append(("standardisation", StandardScaler()))

        steps.append(("classifier", classification_model))

        pipeline = Pipeline(steps)

        # We compute scores using Grid Search on the parameter grid to do model selection with best
        # hyperparameters
        cv = StratifiedKFold(n_splits=5, 
                             shuffle = True, 
                             random_state=42)
        
        if ((embedding_name in params_grid) or (classification_name in params_grid)) and (embedding_name != "FastText"):

            selected_params_grid = {
                **params_grid.get(embedding_name, {}),
                **params_grid.get(classification_name, {})
            }

            grid_search = RandomizedSearchCV(
                pipeline,
                param_distributions=selected_params_grid,
                cv=cv,
                scoring="accuracy",
                n_jobs=-1,
                verbose=0,
                n_iter = 5
            )

            grid_search.fit(X_train, y_train)
            
            score = grid_search.best_score_
            dict_params[embedding_name][classification_name] = grid_search.best_params_

        else:

            scores = cross_val_score(
                pipeline,
                X_train, 
                y_train,
                cv=cv,
                scoring="accuracy"
            )

            score = np.mean(scores)
            
        dict_scores[embedding_name][classification_name] = score

In [ ]:
pprint(dict_scores)

In [ ]:
best_params = dict_params["TF-IDF"]["Random Forest"]
best_params

# Model evaluation

In [ ]:
pipeline = Pipeline([
    ("feature_extraction", TfidfVectorizer()),
    ("classifier", RandomForestClassifier())
])
pipeline.set_params(**best_params)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
confusion_matrix(y_test, y_pred)

In [ ]:
joblib.dump(pipeline,
            "../models/ml_model.joblib")